# Practice: Automatic Differentiation
JAX를 통해 연습하겠습니다.

In [ ]:
import jax  # AD library JAX를 import 합니다.
import jax.numpy as jnp  # jnp는 numpy와 "거의" 같습니다.

## 속도 비교
[여기](speed-forward-vs-reverse)에서 설명되었듯이,
{math}`d_\mathrm{in} < d_\mathrm{out}`인 경우 forward mode가 빠르고
반대의 경우 reverse mode가 빠릅니다.

먼저 {math}`d_\mathrm{in} < d_\mathrm{out}`인 경우를 테스트 하겠습니다.

In [ ]:
def f(x: float): 
    # scalar를 받아서 [1, x, ..., x^9999]를 리턴하는 함수, d_in < d_out
    return jnp.power(x, jnp.arange(10000))

f_fwd = jax.jacfwd(f)
f_rev = jax.jacrev(f)
x = 1.0

함수 `f`의 경우 $d_\mathrm{in} = 1 \ll d_\mathrm{out} = 10^4$ 이므로, forward mode로 미분한 함수 `f_fwd`가 reverse mode로 미분한 함수 `f_rev`보다 빨라야 합니다.

```{prf:remark}
시간을 잴 함수를 `jax.jit`에 통과시키면 JIT 컴파일이 되고, 계산 속도가 빨리집니다.
```

In [2]:
f_fwd = jax.jit(f_fwd) # compile
f_fwd(x) # warmup
%timeit f_fwd(x).block_until_ready()

4.55 ms ± 599 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
f_rev = jax.jit(f_rev)
f_rev(x)
%timeit f_rev(x).block_until_ready()

10.8 ms ± 338 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def g(y: jnp.ndarray):
    # array를 받아서 제곱한 후 평균을 취하는 함수, d_in > d_out
    return (y**2).mean()

g_fwd = jax.jacfwd(g)
g_rev = jax.jacrev(g)
y = jnp.ones((10000,))

`f_fwd`가 약 두배 정도 빠른 것을 볼 수 있습니다.

반면, 함수 `g`의 경우 $d_\mathrm{in} = 10^4 \gg d_\mathrm{out} = 1$ 이므로, reverse mode로 미분한 함수 `g_rev`가 forward mode로 미분한 함수 `g_fwd` 보다 빨라야 합니다.

In [4]:
g_fwd = jax.jit(g_fwd)
g_fwd(y)
%timeit g_fwd(y).block_until_ready()

11.9 ms ± 101 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
g_rev = jax.jit(g_rev)
g_rev(y)
%timeit g_rev(y).block_until_ready()

6.52 ms ± 452 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


`g_rev`가 약 두배 정도 빠른 것을 볼 수 있습니다.

